In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import math

import cv2
from PIL import Image
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
import tensorflow as tf
from tqdm import tqdm

%matplotlib inline

**Reading and printing the shapes of the train and test datasets**

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
print(train_df.shape)
print(test_df.shape)
train_df.head()

In [ ]:
def display_samples(df, columns=4, rows=3):
    fig=plt.figure(figsize=(18, 10))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
        image_id = df.loc[i,'diagnosis']
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(img)
    
display_samples(train_df)

**Since i want to try the classification using InceptionV3, the images in the dataset must be resized to 299x299**

In [ ]:
def resize_image(image_path, desired_size=299):
    im = Image.open(image_path)
    im = im.resize((desired_size, desired_size ), resample=Image.ANTIALIAS)
    
    return im

**Resizing the images**

In [ ]:
N = train_df.shape[0]
x_train = np.empty((N, 299, 299, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(train_df['id_code'])):
    x_train[i, :, :, :] = resize_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png'
    )

In [ ]:
N = test_df.shape[0]
x_test = np.empty((N, 299, 299, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(test_df['id_code'])):
    x_test[i, :, :, :] = resize_image(
        f'../input/aptos2019-blindness-detection/test_images/{image_id}.png'
    )

****loading the target classes into y_train****

In [ ]:
y_train = pd.get_dummies(train_df['diagnosis']).values

print(x_train.shape)
print(y_train.shape)
print(y_train[:10])
print(x_test.shape)


****converting the target classes to multilabel (if the class is 4 [0,0,0,1,0] all three previous classes are also active [1,1,1,1,0]

In [ ]:
y_train_multi = np.empty(y_train.shape, dtype=y_train.dtype)
y_train_multi[:, 4] = y_train[:, 4]

for i in range(3, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_train[:, i], y_train_multi[:, i+1])

print("Original y_train:", y_train.sum(axis=0))
print("Multilabel version:", y_train_multi.sum(axis=0))
print(y_train)
print(y_train_multi)

**splitting the dataset into train and validation**

In [ ]:
print(x_train.shape)
print(y_train_multi.shape)

x_trn, x_val, y_trn, y_val = train_test_split(
    x_train, y_train_multi, 
    test_size=0.2,
    random_state=1738
)

In [ ]:
print(x_trn.shape)
print(x_val.shape)
print(y_trn.shape)
print(y_val.shape)

**Data augmentation using ImageDataGenerator with the parameters:**
* Random zoom of 15%
* Random rotation from -90° to 90°
* Random horizontal and vertical flip 

In [ ]:
BATCH_SIZE = 32

def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,  
        rotation_range=90,
        fill_mode='constant',
        cval=0.,  
        horizontal_flip=True,  
        vertical_flip=True, 
    )

data_generator = create_datagen().flow(x_trn, y_trn, batch_size=BATCH_SIZE, seed=1738)